# Import

In [50]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import math
import os 
import time

In [14]:
# settings
# Set the precision for displaying floating-point numbers
pd.set_option('display.precision', 0)


# Functions

In [20]:
# urls
urls = {
    '戶政常見問答集':'https://www.ris.gov.tw/app/portal/158'
}

In [55]:
# 所有標籤頁

url = urls['戶政常見問答集']
response = requests.get(url)
start = datetime.now()
soup = BeautifulSoup(response.text, "html.parser")

tab_url_list =[]
# tab content
categories = soup.findAll('div',{'class' :'accordion-body accordion-second2-body collapse in'})
category = categories[0].findAll('ul', {'class' : 'nav nav-list'})

print(f'Start retreiving from ... {url}')
for _ in category[0].find_all('a', href=True):
    if '#' not in _['href']:
        tab_url_list.append(_['href'])

print(f'Got {len(tab_url_list)} urls')
end = datetime.now()
print(f'Time : {end-start}')



Start retreiving from ... https://www.ris.gov.tw/app/portal/158
Got 12 urls
Time : 0:00:00.024510


In [124]:
# get qa list
import os

df = []
idx = 0
start = datetime.now()
for url in tab_url_list:
    print(url)
    r = requests.get(url) 
    soup = BeautifulSoup(r.text, "html.parser")

    #https://www.ris.gov.tw/documents/html/8/7/1/158_Q01.html
    file_path = soup.find_all('iframe')[0]['src']

    # tab content
    r = requests.get('https://www.ris.gov.tw/'+soup.find_all('iframe')[0]['src']) 
    soup = BeautifulSoup(r.text, "html.parser")
    for _ in soup.find_all('a',href='javascript:void(0);'):
        qa={}
        qa['category'] = soup.find_all('h1')[0].text
        # Split the path and filename
        directory, filename = os.path.split(file_path)
        new_filename = _['onclick'].split('window.location.href=')[1].strip(';\'')

        # Create the new file path
        new_file_path = os.path.join(directory, new_filename)

        # request to the qa page
        r = requests.get('https://www.ris.gov.tw/'+new_file_path) 
        soup = BeautifulSoup(r.text, "html.parser")
        if len(soup.find_all('legend')[0].text)>0:
            qa['question'] = soup.find_all('legend')[0].text
            qa['answer']=''
            for _ in soup.find_all('li'):
                qa['answer']+=_.text.strip().replace('\n','').replace('\t','').replace('\r','')
        
        if len(qa):
            df.append(qa)
end = datetime.now()
print(f'Time : {end-start}')

https://www.ris.gov.tw/app/portal/158
https://www.ris.gov.tw/app/portal/150
https://www.ris.gov.tw/app/portal/149
https://www.ris.gov.tw/app/portal/741
https://www.ris.gov.tw/app/portal/144
https://www.ris.gov.tw/app/portal/145
https://www.ris.gov.tw/app/portal/151
https://www.ris.gov.tw/app/portal/153
https://www.ris.gov.tw/app/portal/154
https://www.ris.gov.tw/app/portal/155
https://www.ris.gov.tw/app/portal/156
https://www.ris.gov.tw/app/portal/659
Time : 0:00:30.962317


In [125]:
pd.DataFrame(df)

,category,question,answer
0,常見問答集,Q01.子女從姓及改姓有那些規定？又應如何辦理登記？,一、配合民法96年5月23日修正公布，戶政事務所辦理新生兒辦理出生登記時從姓之處理原則：父母...
1,常見問答集,Q02.改名有那些規定？又應如何辦理登記？,一、姓名符合下列情形之一者，當事人可向全國任一戶政事務所申請改名：同時在一公民營事業機構、機...
2,常見問答集,Q03.如何辦理認領登記？,認領登記，以認領人為申請人，認領人不為申請時，以被認領人為申請人，並準備下列文件至任一戶政事...
3,常見問答集,Q04.承租人已搬走但戶籍未遷出，可否將其戶籍遷出？,一、按行政法院56年判字第60號判例：「遷徙係事實行為，遷徙登記自應依事實認定之。」次按戶籍...
4,常見問答集,Q05.如何辦理初設戶籍登記？,一、申請人:（一）當事人為未成年人：由法定代理人（父母雙方）、戶長。（二）當事人為成年人：本...
...,...,...,...
75,指定送達地址申請作業,Q01. 辦理指定送達地址目的為何？,為落實簡政便民，提升服務品質，戶政機關行政處分或告知單等文件之送達，應優先寄送民眾指定送達地...
76,指定送達地址申請作業,Q02. 我可以隨時上網申辦指定送達地址嗎？,可以，本系統全年無休24小時全天開放，提供民眾快速便捷且安全無虞的申請管道，亦可以到各戶政事...
77,指定送達地址申請作業,Q03. 我該如何申辦指定送達地址呢？,請利用具自然人憑證ＩＣ卡相容讀卡機之個人電腦上網至「內政部戶政司全球資訊網」網站（https...
78,指定送達地址申請作業,Q04. 申辦指定送達地址需要付規費嗎？,目前查詢及申辦指定送達地址均不需要支付規費。


In [128]:
# save 
pd.DataFrame(df).to_csv('./data/qa.csv', index=False)